In [125]:
from random import shuffle, randint
from time import sleep
from collections import Counter
FLAG_CHAR = chr(9872) # 'F'
UNREVEALED_CHAR = "." # "#" # " "
MINE_CHAR = "*"
EMPTY_CHAR = "-"
BOARD_XRANGE = 30
BOARD_YRANGE = 20
MINE_COUNT = int(0.15*(BOARD_XRANGE*BOARD_YRANGE))

In [61]:

class Board:
    """Minesweeper board class"""

    def __init__(self, size, mineCount):
        """Minesweeper board constructor"""
        self.size = size
        self.mineCount = mineCount
        minearr = [1 if i < mineCount else 0 for i in range(size['x'] * size['y'])]
        shuffle(minearr)
        self.mineBoard = [minearr[size['x'] * i:size['x'] * (i + 1)] for i in range(size['y'])]
        self.gameBoard = [[0 for i in range(size['x'])] for j in range(size['y'])]
        self.flagBoard = [[0 for i in range(size['x'])] for j in range(size['y'])]
        self.revealed = [[0 for i in range(size['x'])] for j in range(size['y'])]
        self.gameStarted = False
        self.gameOver = False
        self.gameVictory = False
    def makegameboard(self):
        for j in range(self.size['y']):
            for i in range(self.size['x']):
                self.gameBoard[j][i] = 9 if self.mineBoard[j][i] == 1 else self.countAround(self.mineBoard, j, i)
    def display(self):
        MAX_NUMB_LENGTH = 3
        horizontal_indexes = [("#"*(MAX_NUMB_LENGTH+1))+''.join(str((j//10**i)%10) for j in range(1,self.size['x']+1)) for i in reversed(range(MAX_NUMB_LENGTH))]
        horizontal_indexes += ["#"*(MAX_NUMB_LENGTH)+'+'+'-'*self.size['x']]
        lines = ["".join(str(self.gameBoard[n][m]) if i != 0 else UNREVEALED_CHAR if self.flagBoard[n][m] == 0 else FLAG_CHAR for m, i in enumerate(j)) for n, j in enumerate(self.revealed)]
        lines = [("{:0"+str(MAX_NUMB_LENGTH)+"}").format(n+1)+'|'+i for n,i in enumerate(lines)]
        print("\n".join(horizontal_indexes+lines))
    def countAround(self, matrix, y, x):
        """Count mines around given location"""
        return sum(sum(i for i in j[x - 1 if x - 1 > 0 else 0:x + 2 if x + 2 < len(j) else len(j)]) for j in
                   matrix[y - 1 if y - 1 > 0 else 0:y + 2 if y + 2 < len(matrix) else len(matrix)])
    def difficulty(self):
        return int(100 * self.mineCount / (self.size['x'] * self.size['y']))
    def getDisplay(self):
        return "\n".join("".join(
            str(self.gameBoard[n][m]) if i != 0 else UNREVEALED_CHAR if self.flagBoard[n][m] == 0 else FLAG_CHAR for
            m, i in enumerate(j)) for n, j in enumerate(self.revealed))
    def checkVictoryCondition(self):
        """Checking thats all minefield is revealed"""
        self.gameVictory = all(
            all(self.mineBoard[j][i] + self.revealed[j][i] == 1 for i in range(len(self.mineBoard[0]))) for j in
            range(len(self.mineBoard)))
    def checkMine(self, y, x):
        """Check board element for mine"""
        # check correctness of input data
        if not (y < self.size['y'] and x < self.size['x'] and x > -1 and y > -1):
            return
        # do nothing if calculated already
        if self.revealed[y][x] == 1:
            return
        # generate game board on first move
        if not self.gameStarted:
            self.makegameboard()
            self.gameStarted = True
            # move mine if find mine on first move
            if self.mineBoard[y][x] == 1:
                for n, i in enumerate(self.mineBoard):
                    if 0 in i:
                        self.mineBoard[n][i.index(0)] = 1
                        self.checkMine(y, x)
                self.mineBoard[y][x] = 0
        # end game if found mine
        if self.gameBoard[y][x] == 9:
            self.gameOver = True
            return
        if self.gameBoard[y][x] == 0:
            self.revealed[y][x] = 1
            self.flagBoard[y][x] = 0
            for i in range(-1, 2):
                for j in range(-1, 2):
                    if not i == j == 0:
                        self.checkMine(y + i, x + j)
        else:
            self.revealed[y][x] = 1
            self.flagBoard[y][x] = 0
        self.checkVictoryCondition()
    def putFlag(self, y, x):
        """Put flag on board or remove if already placed"""
        self.flagBoard[y][x] = 1 if self.flagBoard[y][x] == 0 and self.revealed[y][x] == 0 else 0

In [114]:

class AiAgent:
    def __init__(self, board):
        self.board = board
        self.inner_state = 0

    def compareUnrevealed(self, matrix, y, x):
        """Compare unrevealed dots around given dot with dot's number"""
        flags = 0
        unrevealed = 0
        unrevealeddots = []
        if matrix[y][x].isdigit():
            number = int(matrix[y][x])
        else:
            return False, []
        for j in range(y - 1 if y - 1 > 0 else 0, y + 2 if y + 2 < len(matrix) else len(matrix)):
            for i in range(x - 1 if x - 1 > 0 else 0, x + 2 if x + 2 < len(matrix[0]) else len(matrix[0])):
                if matrix[j][i] == FLAG_CHAR:
                    flags += 1
                elif matrix[j][i] == UNREVEALED_CHAR:
                    unrevealed += 1
                    unrevealeddots.append((j, i))
                elif matrix[j][i].isdigit():
                    continue
                else:
                    print("Something strange happening in ai agent")
        return number - unrevealed - flags == 0, unrevealeddots

    def findbombs(self):
        boarddata = self.board.getDisplay()
        matrix = boarddata.split("\n")
        dots = []
        for j in range(len(matrix)):
            for i in range(len(matrix[0])):
                checkresult, tempdots = self.compareUnrevealed(matrix, j, i)
                if checkresult: dots += tempdots
        return list(set(dots))

    def compareFlags(self, matrix, y, x):
        """Compare flags around given dot with dot's number"""
        flags = 0
        unrevealed = 0
        unrevealeddots = []
        if matrix[y][x].isdigit():
            number = int(matrix[y][x])
        else:
            return False, []
        for j in range(y - 1 if y - 1 > 0 else 0, y + 2 if y + 2 < len(matrix) else len(matrix)):
            for i in range(x - 1 if x - 1 > 0 else 0, x + 2 if x + 2 < len(matrix[0]) else len(matrix[0])):
                if matrix[j][i] == FLAG_CHAR:
                    flags += 1
                elif matrix[j][i] == UNREVEALED_CHAR:
                    unrevealed += 1
                    unrevealeddots.append((j, i))
                elif matrix[j][i].isdigit():
                    continue
                else:
                    print("Something strange happening in ai agent")
        return number - flags == 0, unrevealeddots

    def findsafespots(self):
        # if mines count == flag count -> unrevealed around is mineless
        boarddata = self.board.getDisplay()
        matrix = boarddata.split("\n")
        dots = []
        for j in range(len(matrix)):
            for i in range(len(matrix[0])):
                checkresult, tempdots = self.compareFlags(matrix, j, i)
                if checkresult: dots += tempdots
        return list(set(dots))
    
    def random_pick(self):
        brd = self.board.getDisplay().split("\n")
        while True:
            y = randint(0, self.board.size["y"] - 1)
            x = randint(0, self.board.size["x"] - 1)
            if brd[y][x] == UNREVEALED_CHAR:
                break
        return y, x
    #todo:
    #Smart sampling
    #1.A = Take all revealed non-mine tiles touching unrevealed tiles
    #2.For A tiles:
    #2.1.Place on all touching unrevealed tiles probability caluated by (tile counter - found mines around)/unrevealed tiles around
    #2.2.Use maximum of probabilities if more than one found
    #3.For other unrevealed tiles use probab mines left/tiles unrevealed
    def auto_play_iter(self):
        #print('\t',self.inner_state)
        if self.inner_state == 0:
            tempdisp = self.board.getDisplay()
            dots = self.findbombs()
            for j in dots:
                self.board.putFlag(*j)
            self.inner_state = 1
            self.tempdisp = tempdisp
            return 'Finding bombs',len(dots)>0
        elif self.inner_state == 1:
            dots = self.findsafespots()
            for j in dots:
                self.board.checkMine(*j)
            self.inner_state = 2
            return 'Finding safe spots',len(dots)>0
        elif self.inner_state == 2:
            if self.tempdisp == self.board.getDisplay():#if no change detected
                y, x = self.random_pick()
                self.board.checkMine(y, x)
                #print('\t\t+',self.inner_state)
                return 'Random mine check',True
            else:
                #print('\t\t-',self.inner_state)
                self.inner_state = 0
                return 'Random mine check',False
        else:
            raise Exception('Lol wtf')

In [119]:

def console_interface():
    a = Board({'x': BOARD_XRANGE, 'y': BOARD_YRANGE}, MINE_COUNT)
    b = AiAgent(a)
    #Main game loop
    #If input:
    #0 - find bombs bot mode
    #1 - find safe spots bot mode
    #2 - activate auto bot mode
    #{x} {y} - checks (x,y) for mine
    #{x} {y} {m}:
    #if m = 0 - checks (x,y) for mine
    #if m = 1 - puts flag on (x,y)
    while not a.gameOver and not a.gameVictory:
        a.display()
        arr = list(map(int, input("input coords:\n").split()))
        if len(arr) == 3:#reveal commands
            x, y, mode = arr
        elif len(arr) == 1:#bot commands
            if arr[0] == 0:
                dots = b.findbombs()
                print(dots)
                for i in dots: a.putFlag(*i)
                continue
            elif arr[0] == 1:
                dots = b.findsafespots()
                print(dots)
                for i in dots: a.checkMine(*i)
                continue
            else:#auto bot play
                while not a.gameOver and not a.gameVictory:
                    stage, isShowing = b.auto_play_iter()
                    sleep(0.5)
                    if isShowing:
                        cnt = Counter(a.getDisplay())
                        print('\n'+stage+':','({}/{} bombs found)'.format(cnt[FLAG_CHAR],MINE_COUNT))
                        a.display()
                print("\nFinal state:")
                a.display()
                break
        else:
            x, y = arr
            mode = 0
        x-=1
        y-=1
        if mode == 0:
            a.checkMine(y, x)
        elif mode == 1:
            a.putFlag(y, x)
        #elif mode == 2:
        #    b.compareFlags(a.getDisplay().split("\n"), y, x)
    print("Victory" if a.gameVictory else "Game Over")
    if (a.gameOver):    
        print("Mine positions:")
        for i in a.mineBoard:
            print("".join(MINE_CHAR if j==1 else EMPTY_CHAR for j in i))

In [127]:
console_interface()

####000000000000000000000000000000
####000000000111111111122222222223
####123456789012345678901234567890
###+------------------------------
001|..............................
002|..............................
003|..............................
004|..............................
005|..............................
006|..............................
007|..............................
008|..............................
009|..............................
010|..............................
011|..............................
012|..............................
013|..............................
014|..............................
015|..............................
016|..............................
017|..............................
018|..............................
019|..............................
020|..............................
input coords:
2

Random mine check: (0/90 bombs found)
####000000000000000000000000000000
####000000000111111111122222222223
####123456789012345678901234567890
